In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.python.ops import math_ops
import random
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm
from matplotlib.ticker import LinearLocator

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses
from keras import backend as K


random.seed(10)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
data = pd.read_csv("MRdata.csv")
X_train = data.iloc[:, 0:1]
y_train = data.iloc[:, 3:4]

y_train_w_noise = data.iloc[:, 4:5]

In [ ]:
data

In [ ]:
loss_tracker = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")


class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            
            y_pred = test_tm_pred(self, x)
#             y_pred = self(x, training=True)
            # Compute our own loss
            loss = keras.losses.mean_squared_error(y, y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        loss_tracker.update_state(loss)
        mae_metric.update_state(y, y_pred)
        return {"loss": loss_tracker.result(), "mae": mae_metric.result()}

    @property
    def metrics(self):
                # We list our `Metric` objects here so that `reset_states()` can be
                # called automatically at the start of each epoch
                # or at the start of `evaluate()`.
                # If you don't implement this property, you have to call
                # `reset_states()` yourself at the time of your choosing.
        return [loss_tracker, mae_metric]

In [ ]:
input_1 = Input(shape=(1,))
output_1 = Dense(1)(input_1)

input_2 = Input(shape=(1,))
output_2 = Dense(1)(input_2)

output_1 = Dense(1)(output_1)
output_2 = Dense(1)(output_2)

concat_2 = tf.keras.layers.Concatenate()([output_1, output_2])
output_2 = Dense(1)(concat_2)
model1 = CustomModel(inputs=[input_1, input_2], outputs=[output_2])

layerCount = len(model1.layers)
lastLayer = layerCount - 1;

lastWeights = model1.layers[lastLayer].get_weights()
lastWeights[0][0] = 1
lastWeights[0][1] = 1
lastWeights[1][0] = 0

model1.layers[lastLayer].set_weights(lastWeights)

model1.layers[lastLayer].trainable = False

opt = tf.keras.optimizers.RMSprop(0.02)
# We don't passs a loss or metrics here.
model1.compile(optimizer=opt, loss="mse", metrics=["mae"])

In [ ]:
# test tm pred
def test_tm_pred(self, x):
        temp_0 = x[0]
        temp_1 = x[1]
        
        x_0 = x[0]**2 + 2/x[0]
        x_1 = 2*x[1] + 1/x[1]/x[1]
        num_step = 30
        increment_0 = x_0/num_step
        increment_1 = x_1/num_step
        start_0 = x_0*0
        start_1 = x_1*0
        start = [start_0, start_1]
        
        layers4 = self.layers[4]
        layers5 = self.layers[5]
        
        ans_0 = layers4(start[0])
        ans_1 = layers5(start[1])
    
        ans1 = 0
        ans2 = 0
        
        for i in range(num_step):
            _x = start
            with tf.GradientTape() as t:
                t.watch(_x)
                _y = self(_x)

            dy_dx = t.gradient(_y, _x)   
    ### x2
            ans1 = ans1 + dy_dx[0]*dy_dx[0]*increment_0
            ans2 = ans2 + dy_dx[1]*dy_dx[1]*increment_1
    ### x2
    ### soft
#         ans1 = ans1 + tf.math.log(tf.math.pow(2.0, dy_dx[0]) + 1)/tf.math.log(2.0)*increment_0
#         ans2 = ans2 + tf.math.log(tf.math.pow(2.0, dy_dx[1]) + 1)/tf.math.log(2.0)*increment_1
    ### soft
            start_0 = start_0 + increment_0
            start_1 = start_1 + increment_1
            start = [start_0, start_1]
#         return ans + ans1*2*(temp_0**2 - 1/temp_0) + ans2*2*(1/temp_1/temp_1 - temp_1)
#         2*temp_0*(temp_0-1/temp_0/temp_0)*(ans_increment1+1/temp_0*ans_increment2)
        return 2*temp_0*(temp_0-1/temp_0/temp_0)*(ans1+1/temp_0*ans2) + ans_1*2*(temp_0 - 1/temp_0/temp_0) + ans_0*2*(temp_0**2 - 1/temp_0)

In [ ]:
model1.fit([X_train.to_numpy(), X_train.to_numpy()], y_train.to_numpy(), epochs=2000)

In [ ]:
# test tm pred
def tm_pred(model, x):
        temp_0 = x[0]
        temp_1 = temp_0
        
        x[0] = x[0]**2 + 2/x[0]
        x[1] = 2*x[1] + 1/x[1]/x[1]
        
        num_step = 10
        increment_0 = x[0]/num_step
        increment_1 = x[1]/num_step
        start_0 = x[0]*0
        start_1 = x[1]*0
        start = [start_0, start_1]
        ans_0 = model(start, training=True)
        
        ##
        layerCount = len(model.layers)
        lastLayer = layerCount - 1;

        ## 
        lastWeights = model.layers[lastLayer].get_weights()
        lastWeights[0][0] = 1
        lastWeights[0][1] = 0

        model.layers[lastLayer].set_weights(lastWeights)

        ans_0 = model(start, training=True)
        
        ## 
        lastWeights[0][0] = 0
        lastWeights[0][1] = 1

        model.layers[lastLayer].set_weights(lastWeights)

        ans_1 = model(start, training=True)
        
        lastWeights[0][0] = 1
        lastWeights[0][1] = 1

        model.layers[lastLayer].set_weights(lastWeights)

        ##
        values = []
        
        ans_increment1 = 0
        ans_increment2 = 0
        for i in range(num_step):
            _x = start
            _x = [list(_x[0]), list(_x[1])]
            _x = [tf.constant(_x[0]), tf.constant(_x[1])]
            
            with tf.GradientTape() as t:
                t.watch(_x)
                _y = model(_x)

            dy_dx = t.gradient(_y, _x)
                ### x2
            ans_increment1 = ans_increment1 + dy_dx[0]*dy_dx[0]*increment_0
            ans_increment2 = ans_increment2 + dy_dx[1]*dy_dx[1]*increment_1
                ### x2
                ### soft
#             ans_increment1 = ans_increment1 + tf.math.log(tf.math.pow(2.0, dy_dx[0]) + 1)/tf.math.log(2.0)*increment_0
#             ans_increment2 = ans_increment2 + tf.math.log(tf.math.pow(2.0, dy_dx[1]) + 1)/tf.math.log(2.0)*increment_1
                ### soft
                
            values.append(ans_increment1)
            
            start_0 = start_0 + increment_0
            start_1 = start_1 + increment_1
            start = [start_0, start_1]
#         ans_increment = ans_increment1*2*(temp_0**2 - 1/temp_0) + ans_increment2*2*(1/temp_1/temp_1 - temp_1)
        ans_increment = 2*temp_0*(temp_0-1/temp_0/temp_0)*(ans_increment1+1/temp_0*ans_increment2)
        ans_increment = tf.reshape(ans_increment, ans_0.shape)
        ans_increment = tf.dtypes.cast(ans_increment, tf.float32)
#         return ans + ans_increment
        return ans_increment + ans_1*2*(temp_0 - 1/temp_0/temp_0) + ans_0*2*(temp_0**2 - 1/temp_0)

In [ ]:
predict1 = tm_pred(model1, [X_train.to_numpy(), X_train.to_numpy()])

In [ ]:
plt.plot(X_train.to_numpy(), y_train, label='train data')
plt.plot(X_train.to_numpy(), predict1, label='fitted convex model')
plt.legend()
# plt.title('2_l_2_n_exp_combined_NN')
# plt.xlabel('I1')

# plt.xlabel('lambda')
# plt.ylabel('Cauchy stress')
plt.savefig('result/correct_MR/2_l_2_n_square_MR.png')
plt.show()

In [ ]:
np.savetxt("MATLAB_figure_data/MR/square/y_train.csv", y_train, delimiter=",")
np.savetxt("MATLAB_figure_data/MR/square/y_prediction.csv", predict1, delimiter=",")

In [ ]:
model1.fit([X_train.to_numpy(), X_train.to_numpy()], y_train_w_noise.to_numpy(), epochs=2000)

In [ ]:
predict1 = tm_pred(model1, [X_train.to_numpy(), X_train.to_numpy()])

In [ ]:
plt.scatter(X_train.to_numpy(), y_train_w_noise, label='train data', s=5, c='r', alpha=1)
plt.plot(X_train.to_numpy(), predict1, label='fitted convex model')
plt.legend()
# plt.title('2_l_2_n_exp_combined_NN')
# plt.xlabel('I1')

# plt.xlabel('lambda')
# plt.ylabel('Cauchy stress')
plt.savefig('result/correct_MR/2_l_2_n_square_MR_noise.png')
plt.show()

In [ ]:
np.savetxt("MATLAB_figure_data/MR/square/y_train_noise.csv", y_train_w_noise, delimiter=",")
np.savetxt("MATLAB_figure_data/MR/square/y_prediction_noise.csv", predict1, delimiter=",")